In [2]:
import pandas as pd
from pathlib import Path


In [3]:
df.head()

,care_home_name,capacity,available_beds
0,Abbeygate Care,17,1
1,Abbeymere,18,2
2,Allenbrook Nursing,36,6
3,Amberley Care Home,25,4
4,Arcare For Forte,9,0


In [5]:
home_dir = Path.home()
home_dir

PosixPath('/Users/muraliveerabahu')

In [6]:
cwd = Path.cwd()
cwd

PosixPath('/Users/muraliveerabahu/Documents/03_Bootcamp/Challenges/Project-3-Care-Home-Team-5/code/pages')

In [7]:
joined_path = cwd / '..' / '..' / 'data' / 'nhomes_capacity.xlsx'
joined_path

PosixPath('/Users/muraliveerabahu/Documents/03_Bootcamp/Challenges/Project-3-Care-Home-Team-5/code/pages/../../data/nhomes_capacity.xlsx')

In [8]:
dfs = pd.read_excel(joined_path)
dfs.head()

,care_home_name,capacity,available_beds
0,Abbeygate Care,17,1
1,Abbeymere,18,2
2,Allenbrook Nursing,36,6
3,Amberley Care Home,25,4
4,Arcare For Forte,9,0


In [9]:
# read the excel file
path_to_excel = Path(r'/Users/muraliveerabahu/Documents/03_Bootcamp/Challenges/Project-3-Care-Home-Team-5/data/nhomes_capacity.xlsx')
dfx = pd.read_excel(path_to_excel)

In [15]:
input_address = 'Russells Hall Hospital, Pensnett Road, Dudley, DY1 2HQ'
input_age = 68

In [4]:
p = Path.cwd() / '..' / 'data' / 'CareHome-data.csv'
df = pd.read_csv(p)
df.head(100)

,Care Home Name,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \nNursing,Capacity,Post Code,Coordinates,Latitude,Longitude,Rating
0,Voyage care,"48 Cedar Road (Voyage) Dudley, DY1 4HW",48CedarRoad@voyagecare.\ncom,01384 241877,Under 65,Learning Disability \nMental Health Condition ...,No,9.0,DY1 4HW,"[52.5, -2.08333]",52.500000,-2.083330,No published rating
1,Abbeygate Care,Abbeygate Care Centre 2 Leys Road Brockmoor Br...,abbeygatecare1@gmail.com,01384 571295,65+,Dementia \nMental Health Condition \nOld Age \...,No,17.0,DY5 3UR,"[52.4857514, -2.1341733]",52.485751,-2.134173,No published rating
2,Abbeymere,Abbeymere 12 Eggington Road Wollaston Stourbri...,abbeymere@karelink.co.uk,01384 395195,65+,Dementia \nMental Health Condition \nOld Age \...,No,18.0,DY8 2QJ,"[52.4584, -2.16295]",52.458400,-2.162950,No published rating
3,Allenbrook Nursing,Allenbrook Nursing Home 209 Spies Lane Halesow...,manager@allenbrooknursing\nhome.co.uk,0121 422 5844,55+,Dementia \nMental Health Condition \nNo Medica...,Yes,36.0,B62 9SJ,"[52.4613003, -2.0139452]",52.461300,-2.013945,No published rating
4,Amberley Care Home,Amberley Care Home 481-483 Stourbridge Road Ha...,amberleycarehome@hotmail\n.co.uk,01384 482365,65+,Dementia \nOld Age,No,25.0,DY5 1LB,"[52.4927718, -2.1164204]",52.492772,-2.116420,No published rating


In [6]:
from config import google_key
import googlemaps
from pathlib import Path
import pandas as pd


In [8]:


# Read the CSV file
path_to_csv = Path.cwd() / '..' / 'data' / 'CareHome-data.csv'
df = pd.read_csv(path_to_csv)

path_to_csv_capacity = Path.cwd() / '..' / 'data' / 'nhomes_capacity.xlsx'
df_capacity = pd.read_excel(path_to_csv_capacity)

# Create a new dataframe for distance
df_dist = pd.DataFrame()
df_dist_disp = pd.DataFrame()

# Find the distance between the input address and the care homes

# Copying columns from the original dataframe
df_dist['Care_Home'] = df['Care Home Name']
df_dist['Address'] = df['Name & Address']


In [11]:
# Create a googlemaps client
gmaps = googlemaps.Client(key=google_key)

# Loop through the dataframe and find the distance
for i in range(len(df_dist)):
    ch_dist = gmaps.distance_matrix(input_address, df_dist.loc[i,'Address'], mode='driving')
    df_dist.loc[i, 'Driving_Distance_Num'] = ch_dist['rows'][0]['elements'][0]['distance']['value']
    df_dist.loc[i, 'Driving_Distance'] = ch_dist['rows'][0]['elements'][0]['distance']['text']




In [13]:
# Sort acending order df_dist by driving distance
df_dist_disp = df_dist.sort_values(by='Driving_Distance_Num', ascending=True)

df_dist_disp['Capacity'] = df_capacity['available_beds']
# Splitting the Age Range column into 'Min Age' and 'Max Age'
df_dist_disp['minAge'] = df['Age Range'].str.extract('(\d{2})')
df_dist_disp['maxAge'] = df['Age Range'].str.extract('-(\d{2})')

# Fill NaN values with 0
df_dist_disp['minAge'] = df_dist_disp['minAge'].fillna(0)
df_dist_disp['maxAge'] = df_dist_disp['maxAge'].fillna(0)

# Change the data type of 'minAge' and 'maxAge' to integer
df_dist_disp['minAge'] = df_dist_disp['minAge'].astype(int)
df_dist_disp['maxAge'] = df_dist_disp['maxAge'].astype(int)

In [16]:
# Filter if input age is >= minAge
df_dist_disp = df_dist_disp[(df_dist_disp['minAge'] <= input_age)]

# Filter if input age is <= maxAge only if maxAge is not 0
df_dist_disp = df_dist_disp[(df_dist_disp['maxAge'] == 0) | (df_dist_disp['maxAge'] >= input_age)]

# Filter if capacity is greater than 0
df_dist_disp = df_dist_disp[(df_dist_disp['Capacity'] > 0)]

In [18]:
df_dist_disp = df_dist_disp.drop(columns=['Driving_Distance_Num', 'minAge', 'maxAge'])

In [20]:
df_dist_disp.head()

,Care_Home,Address,Driving_Distance,Capacity
63,Stoneleigh House,Stoneleigh House 166-168 Stourbridge Road Dudl...,0.8 km,3
60,Russell Court,Russell Court Overfield Road Dudley DY1 2NY,1.0 km,4
9,Beatrice House,Beatrice House 25 Bell Street Pensnett Brierle...,1.1 km,1
10,Belvidere,Belvidere 41-43 Stourbridge Road Holly Hall Du...,1.2 km,2
59,Rosewood Care Home,Rosewood Care Home 139-143 High Street Pensnet...,1.6 km,5


In [23]:
def highlight_rating(val):
    if 'House' in val:
        return 'color: green'
    elif 'Court' in val:
        return 'color: orange'
    
    
df_dist_disp.style.applymap(highlight_rating, subset=['Care_Home'])

,Care_Home,Address,Driving_Distance,Capacity
63,Stoneleigh House,Stoneleigh House 166-168 Stourbridge Road Dudley DY1 2ER,0.8 km,3
60,Russell Court,Russell Court Overfield Road Dudley DY1 2NY,1.0 km,4
9,Beatrice House,Beatrice House 25 Bell Street Pensnett Brierley Hill DY5 4HG,1.1 km,1
10,Belvidere,Belvidere 41-43 Stourbridge Road Holly Hall Dudley DY1 2DH,1.2 km,2
59,Rosewood Care Home,Rosewood Care Home 139-143 High Street Pensnett Brierley Hill DY5 4EA,1.6 km,5
44,Nelson House,Nelson House 1-3 Nelson Road Dudley DY1 2AG,2.2 km,3
29,Himley Manor,Himley Manor 133 Himley Road Dudley DY1 2QF,2.5 km,10
67,Victoria Lodge,Victoria Lodge Bent Street Brierley Hill DY5 2RB,2.6 km,7
68,Village Nursing Home,Village Nursing Home (The) 190 Commonside Brierley Hill DY5 4AA,2.7 km,6
14,Castle View,Castle View 6 Priory Road Dudley DY1 4AD,3.1 km,3


In [24]:
def color_rating(val):
            if 'Home' in val:
                color = 'red'
            else:
                color = 'black'
            return 'color: %s' % color
        
df_dist_disp['Care_Home'] = df_dist_disp['Care_Home'].apply(color_rating)

In [25]:
df_dist_disp.head()

,Care_Home,Address,Driving_Distance,Capacity
63,color: black,Stoneleigh House 166-168 Stourbridge Road Dudl...,0.8 km,3
60,color: black,Russell Court Overfield Road Dudley DY1 2NY,1.0 km,4
9,color: black,Beatrice House 25 Bell Street Pensnett Brierle...,1.1 km,1
10,color: black,Belvidere 41-43 Stourbridge Road Holly Hall Du...,1.2 km,2
59,color: red,Rosewood Care Home 139-143 High Street Pensnet...,1.6 km,5


In [26]:
p = Path.cwd() / '..' / 'data' / 'CareHome-data.csv'
df = pd.read_csv(p)
df.head(100)

,Care Home Name,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \nNursing,Capacity,Post Code,Coordinates,Latitude,Longitude,Rating
0,Voyage care,"48 Cedar Road (Voyage) Dudley, DY1 4HW",48CedarRoad@voyagecare.\ncom,01384 241877,Under 65,Learning Disability \nMental Health Condition ...,No,9.0,DY1 4HW,"[52.5, -2.08333]",52.500000,-2.083330,No published rating
1,Abbeygate Care,Abbeygate Care Centre 2 Leys Road Brockmoor Br...,abbeygatecare1@gmail.com,01384 571295,65+,Dementia \nMental Health Condition \nOld Age \...,No,17.0,DY5 3UR,"[52.4857514, -2.1341733]",52.485751,-2.134173,No published rating
2,Abbeymere,Abbeymere 12 Eggington Road Wollaston Stourbri...,abbeymere@karelink.co.uk,01384 395195,65+,Dementia \nMental Health Condition \nOld Age \...,No,18.0,DY8 2QJ,"[52.4584, -2.16295]",52.458400,-2.162950,No published rating
3,Allenbrook Nursing,Allenbrook Nursing Home 209 Spies Lane Halesow...,manager@allenbrooknursing\nhome.co.uk,0121 422 5844,55+,Dementia \nMental Health Condition \nNo Medica...,Yes,36.0,B62 9SJ,"[52.4613003, -2.0139452]",52.461300,-2.013945,No published rating
4,Amberley Care Home,Amberley Care Home 481-483 Stourbridge Road Ha...,amberleycarehome@hotmail\n.co.uk,01384 482365,65+,Dementia \nOld Age,No,25.0,DY5 1LB,"[52.4927718, -2.1164204]",52.492772,-2.116420,No published rating
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Woodlands Gate Rest,Woodlands Gate Rest Home 12 Dingle Road Pedmor...,woodlands@sclcare.co.uk,01562 885546,60+,Old Age \nPhysical Disability \nSensory Impair...,No,20.0,DY9 0RS,"[52.4434178, -2.1320989]",52.443418,-2.132099,Good
70,Woodview House,Woodview House Nursing Home Waugh Drive Hagley...,woodview@shaw.co.uk,0121 585 6440,65+,C\nMental Health Condition \nOld Age \nCG Comm...,Yes,24.0,B63 1EN,"[52.4356855, -2.0820804]",52.435685,-2.082080,No published rating
71,Wordsley Hall,Wordsley Hall Mill Street Wordsley Stourbridge...,manager.wordsleyhall@min\nstercaregroup.co.uk,01384 571606,65+,"Old Age, Dementia",No,41.0,DY8 5SX,"[52.4780241, -2.1562237]",52.478024,-2.156224,Good
72,Wychbury Care Home,Wychbury Care Home 350-352 Hagley Road Pedmore...,daniel@wychcare.com,01562 885106,65+,", Old Age",No,42.0,DY9 0QY,"[52.4327421, -2.1301692]",52.432742,-2.130169,Good


In [28]:
# Format the rating columm with color
def color_rating(val):
    val = val.lower()
    if 'good' or 'outstanding' in val:
        return 'color: green'
    elif 'requires improvement' in val:
        return 'color: orange'
    elif 'Inadequate' in val:
        return 'color: red'

df.style.applymap(color_rating, subset=['Rating'])

,Care Home Name,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide Nursing,Capacity,Post Code,Coordinates,Latitude,Longitude,Rating
0,Voyage care,"48 Cedar Road (Voyage) Dudley, DY1 4HW",48CedarRoad@voyagecare. com,01384 241877,Under 65,Learning Disability Mental Health Condition Physical Disabilities Sensory Impairment Substance Misuse Younger Adults,No,9.000000,DY1 4HW,"[52.5, -2.08333]",52.500000,-2.083330,No published rating
1,Abbeygate Care,Abbeygate Care Centre 2 Leys Road Brockmoor Brierley Hill DY5 3UR,abbeygatecare1@gmail.com,01384 571295,65+,Dementia Mental Health Condition Old Age Physical Disability Sensory Impairment,No,17.000000,DY5 3UR,"[52.4857514, -2.1341733]",52.485751,-2.134173,No published rating
2,Abbeymere,Abbeymere 12 Eggington Road Wollaston Stourbridge DY8 2QJ,abbeymere@karelink.co.uk,01384 395195,65+,Dementia Mental Health Condition Old Age Physical Disability Sensory Impairment,No,18.000000,DY8 2QJ,"[52.4584, -2.16295]",52.458400,-2.162950,No published rating
3,Allenbrook Nursing,Allenbrook Nursing Home 209 Spies Lane Halesowen B62 9SJ,manager@allenbrooknursing home.co.uk,0121 422 5844,55+,Dementia Mental Health Condition No Medical Intervention Old Age Physical Disability Sensory Impairment,Yes,36.000000,B62 9SJ,"[52.4613003, -2.0139452]",52.461300,-2.013945,No published rating
4,Amberley Care Home,Amberley Care Home 481-483 Stourbridge Road Harts Hill Brierley Hill DY5 1LB,amberleycarehome@hotmail .co.uk,01384 482365,65+,Dementia Old Age,No,25.000000,DY5 1LB,"[52.4927718, -2.1164204]",52.492772,-2.116420,No published rating
5,Arcare For Forte,Arcare For Forte 440 Birmingham New Road Bilston WV14 9QB,ksharma@arcarehomes.co.u k,01902 880108,18+,Learning Disability Mental Health Condition Old Age Physical Disability Sensory Impairment Younger Adults,No,9.000000,WV14 9QB,"[52.5481562, -2.094362]",52.548156,-2.094362,No published rating
6,Ashbourne Care Ltd,Ashbourne Care Ltd Lightwood Road Dudley DY1 2RS,ashbourne.m@fshc.co.uk,01384 242200,65+,Dementia Old Age,No,38.000000,DY1 2RS,"[52.517467, -2.101861]",52.517467,-2.101861,No published rating
7,Ashgrove Nursing,Ashgrove Nursing Home 9 Dudley Wood Road Netherton Dudley DY2 0DA,cea@ashgrovecare.com,01384 413913,65+,Dementia Old Age Sensory Impairment,Yes,57.000000,DY2 0DA,"[52.4777428, -2.0859231]",52.477743,-2.085923,Requires improvement
8,Avondale,Avondale 45 Norton Road Norton Stourbridge DY8 2AH,avondaleresthome@hotmail. com,01384 442731,65+,Old Age,No,15.000000,DY8 2AH,"[52.4453886, -2.157669]",52.445389,-2.157669,No published rating
9,Beatrice House,Beatrice House 25 Bell Street Pensnett Brierley Hill DY5 4HG,beatricehouse@alphonsusse rvices.co.uk,01384 482963,Not Stated,Learning Disability,No,3.000000,DY5 4HG,"[52.4809461, -2.12446]",52.480946,-2.124460,No published rating
